# Setup

We begin by importing what we need, and loading a simulation. We rescale the time for convenience so that a simulation time of 1 corresponds to one orbit of the outermost planet. This counts as 1 beat in the midi file so that in 4/4 time the outermost planet will orbit 4 times in each bar.

In [1]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound
filename = "../binaries/trappist.bin"
sim = rebound.Simulation.from_file(filename)
sim.t = 0
ss.rescale_time(sim, sim.particles[-1].P)

Here we choose to add notes every time there is a transit or conjunction, so we add an `EventRecorder` for each of them. We choose to stagger planets in for the MIDI, so we initially set an empty list of target particles for finding the events, and update them later. 

In [2]:
transits = ss.EventRecorder(sim, lambda sim, i: sim.particles[i].y, targets=[])
conjunctions = ss.EventRecorder(sim, lambda sim, i: np.sin(sim.particles[i].theta - sim.particles[i+1].theta), targets=[])

We start by staggering in the planet transits, 4 outer planet orbits at a time, from the outside inward, by changing `transits.targets`.

In [3]:
planets = list(range(sim.N))
for i in range(1,sim.N):
    ps = planets[-i:]
    transits.targets = ps
    print(transits.targets)
    sim.integrate(tmax=sim.t+4)

[7]
[6, 7]
[5, 6, 7]
[4, 5, 6, 7]
[3, 4, 5, 6, 7]
[2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7]


We now stagger in conjunctions starting from the outermost pair, leaving in all the planets for the transits:

In [4]:
planets = list(range(sim.N-1))
for i in range(1,sim.N-1):
    ps = planets[-i:]
    conjunctions.targets = ps
    print(conjunctions.targets)
    sim.integrate(tmax=sim.t+4)

[6]
[5, 6]
[4, 5, 6]
[3, 4, 5, 6]
[2, 3, 4, 5, 6]
[1, 2, 3, 4, 5, 6]


# Making the MIDI

We need to set a tempo in beats per minute, where 1 beat corresponds to one time unit in the simulation. Here we set bpm=30, which means that there are 2 seconds per simulation time unit = one outer planet orbit. We then make the MIDI file and set the tempo:

In [5]:
from midiutil import MIDIFile

bpm = 30
midifile = MIDIFile(adjust_origin=True)
midifile.addTempo(track=0, time=0, tempo=bpm) 

Now we need to decide on MIDI notes for the transits and conjunctions. We use the `calc_midi_notes` function to scale all notes relative to the outermost (ref_ID=-1) particle, and assign to that planet a MIDI note of 48=C4. See e.g.  http://subsynth.sourceforge.net/midinote2freq.html for a list of MIDI notes. We then manually assign notes to the conjunctions.

Finally, we step through the list of transits and conjunctions, and add the corresponding note for each of them, depending on the transiting planet or conjunction pair, at the time of the event. One can also set the note duration and velocity (volume). Here we add planet transits onto separate channels, and the conjunctions all into one channel above the others.

In [6]:
transit_notes = ss.calc_midi_notes(sim.particles, ref_note=48, ref_ID=-1)
conjunction_notes = [0, 33, 35, 20, 18, 14, 12] #MIDI number choices for TRAPPIST-1 conjunctions, add/subtract elements for other systems

for transit in transits.events:
    midifile.addNote(track=0, channel=transit['target'], pitch=transit_notes[transit['target']], time=transit['time'], duration=1, volume=100)
for conjunction in conjunctions.events:
    midifile.addNote(track=0, channel=sim.N, pitch=conjunction_notes[conjunction['target']], time=conjunction['time'], duration=1, volume=100)

Finally, we write the file:

In [7]:
with open("./trappist.mid", "wb") as f:
    midifile.writeFile(f)